In [ ]:
!git clone https://github.com/vsitzmann/siren.git

fatal: destination path 'siren' already exists and is not an empty directory.


In [ ]:
%cd siren

/content/siren


In [ ]:
from modules import *

In [ ]:
class ConvImgEncoder(nn.Module):
    def __init__(self, channel, image_resolution):
        super().__init__()

        # conv_theta is input convolution
        self.conv_theta = nn.Conv2d(channel, 128, 3, 1, 1)
        self.relu = nn.ReLU(inplace=True)

        self.cnn = nn.Sequential(
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.ReLU(),
            Conv2dResBlock(256, 256),
            Conv2dResBlock(256, 256),
            Conv2dResBlock(256, 256),
            Conv2dResBlock(256, 256),
            nn.Conv2d(256, 256, 1, 1, 0)
        )

        self.relu_2 = nn.ReLU(inplace=True)
        self.fc = nn.Linear(784, 1)

        self.image_resolution = image_resolution

    def forward(self, I):
        o = self.relu(self.conv_theta(I))
        o = self.cnn(o)

        o = self.fc(self.relu_2(o).view(o.shape[0], 256, -1)).squeeze(-1)
        return o

In [ ]:
net_ = ConvImgEncoder(1, 28)

In [ ]:
net_(torch.ones((1, 1, 28, 28))).shape

torch.Size([1, 256])

In [ ]:
'''Modules for hypernetwork experiments, Paper Sec. 4.4
'''

import torch
from torch import nn
from collections import OrderedDict
import modules


class HyperNetwork(nn.Module):
    def __init__(self, hyper_in_features, hyper_hidden_layers, hyper_hidden_features, hypo_module):
        '''

        Args:
            hyper_in_features: In features of hypernetwork
            hyper_hidden_layers: Number of hidden layers in hypernetwork
            hyper_hidden_features: Number of hidden units in hypernetwork
            hypo_module: MetaModule. The module whose parameters are predicted.
        '''
        super().__init__()

        hypo_parameters = hypo_module.meta_named_parameters()

        self.names = []
        self.nets = nn.ModuleList()
        self.param_shapes = []
        for name, param in hypo_parameters:
            self.names.append(name)
            self.param_shapes.append(param.size())

            hn = modules.FCBlock(in_features=hyper_in_features, out_features=int(torch.prod(torch.tensor(param.size()))),
                                 num_hidden_layers=hyper_hidden_layers, hidden_features=hyper_hidden_features,
                                 outermost_linear=True, nonlinearity='relu')
            self.nets.append(hn)

            if 'weight' in name:
                self.nets[-1].net[-1].apply(lambda m: hyper_weight_init(m, param.size()[-1]))
            elif 'bias' in name:
                self.nets[-1].net[-1].apply(lambda m: hyper_bias_init(m))

    def forward(self, z):
        '''
        Args:
            z: Embedding. Input to hypernetwork. Could be output of "Autodecoder" (see above)

        Returns:
            params: OrderedDict. Can be directly passed as the "params" parameter of a MetaModule.
        '''
        params = OrderedDict()
        for name, net, param_shape in zip(self.names, self.nets, self.param_shapes):
            batch_param_shape = (-1,) + param_shape
            params[name] = net(z).reshape(batch_param_shape)
        return params


class NeuralProcessImplicit2DHypernet(nn.Module):
    '''A canonical 2D representation hypernetwork mapping 2D coords to out_features.'''
    def __init__(self, in_features, out_features, image_resolution=None, encoder_nl='sine'):
        super().__init__()

        latent_dim = 256
        self.hypo_net = modules.SingleBVPNet(out_features=out_features, type='sine', sidelength=image_resolution,
                                             in_features=2)
        self.hyper_net = HyperNetwork(hyper_in_features=latent_dim, hyper_hidden_layers=3, hyper_hidden_features=256,
                                      hypo_module=self.hypo_net)
        self.set_encoder = modules.SetEncoder(in_features=in_features, out_features=latent_dim, num_hidden_layers=2,
                                              hidden_features=latent_dim, nonlinearity=encoder_nl)
        print(self)

    def freeze_hypernet(self):
        for param in self.hyper_net.parameters():
            param.requires_grad = False

    def get_hypo_net_weights(self, model_input):
        pixels, coords = model_input['img_sub'], model_input['coords_sub']
        ctxt_mask = model_input.get('ctxt_mask', None)
        embedding = self.set_encoder(coords, pixels, ctxt_mask=ctxt_mask)
        hypo_params = self.hyper_net(embedding)
        return hypo_params, embedding

    def forward(self, model_input):
        if model_input.get('embedding', None) is None:
            pixels, coords = model_input['img_sub'], model_input['coords_sub']
            ctxt_mask = model_input.get('ctxt_mask', None)
            embedding = self.set_encoder(coords, pixels, ctxt_mask=ctxt_mask)
        else:
            embedding = model_input['embedding']
        hypo_params = self.hyper_net(embedding)

        model_output = self.hypo_net(model_input, params=hypo_params)
        return {'model_in':model_output['model_in'], 'model_out':model_output['model_out'], 'latent_vec':embedding,
                'hypo_params':hypo_params}


class ConvolutionalNeuralProcessImplicit2DHypernet(nn.Module):
    def __init__(self, in_features, out_features, image_resolution=28, latent_dim=784, partial_conv=False):
        super().__init__()

        if partial_conv:
            self.encoder = modules.PartialConvImgEncoder(channel=in_features, image_resolution=image_resolution)
        else:
            self.encoder = ConvImgEncoder(channel=in_features, image_resolution=image_resolution)
        self.hypo_net = modules.FCBlock(2, 1, 3, 256, outermost_linear=True, nonlinearity='sine', weight_init=None)
        self.hyper_net = HyperNetwork(hyper_in_features=latent_dim, hyper_hidden_layers=1, hyper_hidden_features=256,
                                      hypo_module=self.hypo_net)

    def forward(self, model_input):
        if model_input.get('embedding', None) is None:
            embedding = self.encoder(model_input['img_sparse'])
        else:
            embedding = model_input['embedding']
        hypo_params = self.hyper_net(embedding)

        model_output = self.hypo_net(model_input["coords"], params=hypo_params)

        return {'model_out': model_output, 'latent_vec': embedding,
                'hypo_params': hypo_params}

    def get_hypo_net_weights(self, model_input):
        embedding = self.encoder(model_input['img_sparse'])
        hypo_params = self.hyper_net(embedding)
        return hypo_params, embedding

    def freeze_hypernet(self):
        for param in self.hyper_net.parameters():
            param.requires_grad = False
        for param in self.encoder.parameters():
            param.requires_grad = False


############################
# Initialization schemes
def hyper_weight_init(m, in_features_main_net):
    if hasattr(m, 'weight'):
        nn.init.kaiming_normal_(m.weight, a=0.0, nonlinearity='relu', mode='fan_in')
        m.weight.data = m.weight.data / 1.e2

    if hasattr(m, 'bias'):
        with torch.no_grad():
            m.bias.uniform_(-1/in_features_main_net, 1/in_features_main_net)


def hyper_bias_init(m):
    if hasattr(m, 'weight'):
        nn.init.kaiming_normal_(m.weight, a=0.0, nonlinearity='relu', mode='fan_in')
        m.weight.data = m.weight.data / 1.e2

    if hasattr(m, 'bias'):
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(m.weight)
        with torch.no_grad():
            m.bias.uniform_(-1/fan_in, 1/fan_in)

In [ ]:
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [ ]:
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=1, shuffle=True)

In [ ]:
print(mnist_trainset[0][0])

<PIL.Image.Image image mode=L size=28x28 at 0x7F942AB6CBD0>


In [ ]:
def get_mgrid(sidelen, dim=2):
    '''Generates a flattened grid of (x,y,...) coordinates in a range of -1 to 1.
    sidelen: int
    dim: int'''
    tensors = tuple(dim * [torch.linspace(-1, 1, steps=sidelen)])
    mgrid = torch.stack(torch.meshgrid(*tensors), dim=-1)
    mgrid = mgrid.reshape(-1, dim)
    return mgrid

In [ ]:
from torch.utils.data import DataLoader, Dataset

class ImageFitting(Dataset):
    def __init__(self, sidelength, img):
        super().__init__()
        print(img.shape)
        self.pixels = img.view(-1, 1)
        self.coords = get_mgrid(sidelength, 2)

    def __len__(self):
        return 1

    def __getitem__(self, idx):    
        if idx > 0: raise IndexError
            
        return self.coords, self.pixels

In [ ]:
import numpy as np
cameraman = ImageFitting(28, torch.Tensor(np.array(mnist_trainset[0][0])))
dataloader = DataLoader(cameraman, batch_size=1, pin_memory=True, num_workers=0)

torch.Size([28, 28])


In [ ]:
img_siren = modules.FCBlock(2, 1, 3, 128, outermost_linear=True, nonlinearity='sine', weight_init=None)
img_siren.cuda()

FCBlock(
  (net): MetaSequential(
    (0): MetaSequential(
      (0): BatchLinear(in_features=2, out_features=128, bias=True)
      (1): Sine()
    )
    (1): MetaSequential(
      (0): BatchLinear(in_features=128, out_features=128, bias=True)
      (1): Sine()
    )
    (2): MetaSequential(
      (0): BatchLinear(in_features=128, out_features=128, bias=True)
      (1): Sine()
    )
    (3): MetaSequential(
      (0): BatchLinear(in_features=128, out_features=128, bias=True)
      (1): Sine()
    )
    (4): MetaSequential(
      (0): BatchLinear(in_features=128, out_features=1, bias=True)
    )
  )
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
total_steps = 1000000 # Since the whole image is our dataset, this just means 500 gradient descent steps.
steps_til_summary = 50

optim = torch.optim.Adam(lr=1e-6, params=net.hyper_net.parameters())

model_input, ground_truth = get_mgrid(28, 2), torch.Tensor(np.array(mnist_trainset[step % len(mnist_trainset)][0])).reshape((784, 1))
model_input, ground_truth = model_input.cuda(), ground_truth.cuda()

for step in range(total_steps):
    
    out = net({'embedding': ground_truth.view((1, 784)).cuda(), 'coords': model_input.cuda()})
    model_output = nn.Sigmoid()(out["model_out"])
    hypo_params = out["hypo_params"]
    all_hypo_params = torch.cat([x.view(-1) for x in list(hypo_params.values())])
    # model_output, coords = img_siren(model_input)    
    loss = ((model_output.cuda() - (ground_truth / 255))**2).mean() + 1000 * ((all_hypo_params.cuda())**2).mean()
    
    if not step % 250:
        print("Step %d, Total loss %0.6f" % (step, loss))

        print(torch.max(model_output))
        print(torch.max(ground_truth))

        fig, axes = plt.subplots(1,3, figsize=(18,6))
        axes[0].imshow(model_output.cpu().view(28,28).detach().numpy())
        axes[1].imshow((ground_truth / 255).cpu().view(28,28).detach().numpy())
        plt.show()

    optim.zero_grad()
    loss.backward()
    optim.step()

NameError: ignored

In [ ]:
class SineLayer(nn.Module):
    # See paper sec. 3.2, final paragraph, and supplement Sec. 1.5 for discussion of omega_0.
    
    # If is_first=True, omega_0 is a frequency factor which simply multiplies the activations before the 
    # nonlinearity. Different signals may require different omega_0 in the first layer - this is a 
    # hyperparameter.
    
    # If is_first=False, then the weights will be divided by omega_0 so as to keep the magnitude of 
    # activations constant, but boost gradients to the weight matrix (see supplement Sec. 1.5)
    
    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        self.init_weights()
    
    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 
                                             1 / self.in_features)      
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0, 
                                             np.sqrt(6 / self.in_features) / self.omega_0)
        
    def forward(self, input):
        return torch.sin(self.omega_0 * self.linear(input))
    
    def forward_with_intermediate(self, input): 
        # For visualization of activation distributions
        intermediate = self.omega_0 * self.linear(input)
        return torch.sin(intermediate), intermediate
    
    
class Siren(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, outermost_linear=False, 
                 first_omega_0=30, hidden_omega_0=30.):
        super().__init__()
        
        self.net = []
        self.net.append(SineLayer(in_features, hidden_features, 
                                  is_first=True, omega_0=first_omega_0))

        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features, 
                                      is_first=False, omega_0=hidden_omega_0))

        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)
            
            with torch.no_grad():
                final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0, 
                                              np.sqrt(6 / hidden_features) / hidden_omega_0)
                
            self.net.append(final_linear)
        else:
            self.net.append(SineLayer(hidden_features, out_features, 
                                      is_first=False, omega_0=hidden_omega_0))
        
        self.net = nn.Sequential(*self.net)
    
    def forward(self, coords):
        coords = coords.clone().detach().requires_grad_(True) # allows to take derivative w.r.t. input
        output = self.net(coords)
        return output, coords        

    def forward_with_activations(self, coords, retain_grad=False):
        '''Returns not only model output, but also intermediate activations.
        Only used for visualizing activations later!'''
        activations = OrderedDict()

        activation_count = 0
        x = coords.clone().detach().requires_grad_(True)
        activations['input'] = x
        for i, layer in enumerate(self.net):
            if isinstance(layer, SineLayer):
                x, intermed = layer.forward_with_intermediate(x)
                
                if retain_grad:
                    x.retain_grad()
                    intermed.retain_grad()
                    
                activations['_'.join((str(layer.__class__), "%d" % activation_count))] = intermed
                activation_count += 1
            else: 
                x = layer(x)
                
                if retain_grad:
                    x.retain_grad()
                    
            activations['_'.join((str(layer.__class__), "%d" % activation_count))] = x
            activation_count += 1

        return activations

In [ ]:
net = ConvolutionalNeuralProcessImplicit2DHypernet(1, 1, image_resolution=28)
net.cuda()

ConvolutionalNeuralProcessImplicit2DHypernet(
  (encoder): ConvImgEncoder(
    (conv_theta): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU(inplace=True)
    (cnn): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2dResBlock(
        (convs): Sequential(
          (0): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (1): ReLU()
          (2): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (3): ReLU()
        )
        (final_relu): ReLU()
      )
      (3): Conv2dResBlock(
        (convs): Sequential(
          (0): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (1): ReLU()
          (2): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (3): ReLU()
        )
        (final_relu): ReLU()
      )
      (4): Conv2dResBlock(
        (convs): Sequential(
        

In [ ]:
class MNISTSirenDataset(torch.utils.data.Dataset):
    def __init__(self, trainMNIST):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.trainMNIST = trainMNIST

    def __len__(self):
        return len(self.trainMNIST)

    def __getitem__(self, idx):
        image = torch.Tensor(np.array(self.trainMNIST[idx][0])).reshape((784, 1))
        coord = get_mgrid(28, 2)

        return image, coord

In [ ]:
loader = DataLoader(MNISTSirenDataset(mnist_trainset), batch_size=32, shuffle=True)

In [ ]:
total_steps = 1000000 # Since the whole image is our dataset, this just means 500 gradient descent steps.
steps_til_summary = 50
batch_size = 8

optim = torch.optim.Adam(lr=5e-5, params=[
                            {'params': net.hyper_net.parameters()},
                            # {'params': net.encoder.parameters()}
                        ])

for e in range(100):
  for i, (img, coords) in enumerate(loader):
      
      # model_input, ground_truth = coords# get_mgrid(28, 2), torch.Tensor(np.array(mnist_trainset[step % len(mnist_trainset)][0])).reshape((784, 1))
      # model_input, ground_truth = model_input.cuda(), ground_truth.cuda()
      img, coords = img.cuda(), coords.cuda()

      out = net_2({'embedding': img.view((32, 784)).cuda() / 255, 'coords': model_input.cuda()})

      model_output = out["model_out"]
      embedding = out["latent_vec"]
      hypo_params = out["hypo_params"]

      all_hypo_params = torch.cat([x.view(-1) for x in list(hypo_params.values())])

      loss = ((model_output.cuda() - (img / 255))**2).mean() + 100 * ((all_hypo_params.cuda())**2).mean() #  + 0.1 * ((embedding.cuda())**2).mean()
      
      if not i % 250:
          print("Step %d, Total loss %0.6f" % (i, loss))

          print(torch.max(model_output))
          print(torch.max(img))

          fig, axes = plt.subplots(1,3, figsize=(18,6))
          axes[0].imshow((model_output[0]).cpu().view(28,28).detach().numpy())
          axes[1].imshow((img / 255)[0].cpu().view(28,28).detach().numpy())
          plt.show()

      optim.zero_grad()
      loss.backward()
      optim.step()

NameError: ignored

In [ ]:
# torch.save(net_2.state_dict(), "mnist_siren")

In [ ]:
# net_2 = ConvolutionalNeuralProcessImplicit2DHypernet(1, 1, image_resolution=28)
# net_2.cuda()
# net_2.load_state_dict(torch.load("/content/drive/My Drive/mnist_flatten_siren"))

<All keys matched successfully>

# MNISTSiren + Pre-trained CNN Classifier

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [ ]:
class MNISTClassifierNet(nn.Module):
    def __init__(self):
        super(MNISTClassifierNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
network = MNISTClassifierNet()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [ ]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.339086
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.305619
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.314029
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.260222
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.289430
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.236755
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.186437
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.169749
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.186659
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.034549
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.098917
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.021254
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.800838
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.744249
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.674355
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.547719
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.243401
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.333821
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.086432
Train Epoch: 1 [12160

In [ ]:
network = network.cuda()
def network_head(x):
  x = F.relu(F.max_pool2d(network.conv1(x), 2))
  x = F.relu(F.max_pool2d(network.conv2_drop(network.conv2(x)), 2))
  x = x.view(-1, 320)
  x = F.relu(network.fc1(x))
  return x

In [ ]:
net_3 = ConvolutionalNeuralProcessImplicit2DHypernet(1, 1, image_resolution=28, latent_dim=50)
net_3.cuda()
# net_3.load_state_dict(torch.load("/content/drive/My Drive/mnist_flatten_siren"))

ConvolutionalNeuralProcessImplicit2DHypernet(
  (encoder): ConvImgEncoder(
    (conv_theta): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU(inplace=True)
    (cnn): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2dResBlock(
        (convs): Sequential(
          (0): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (1): ReLU()
          (2): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (3): ReLU()
        )
        (final_relu): ReLU()
      )
      (3): Conv2dResBlock(
        (convs): Sequential(
          (0): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (1): ReLU()
          (2): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (3): ReLU()
        )
        (final_relu): ReLU()
      )
      (4): Conv2dResBlock(
        (convs): Sequential(
        

In [ ]:
total_steps = 1000000 # Since the whole image is our dataset, this just means 500 gradient descent steps.
steps_til_summary = 50
batch_size = 8

optim = torch.optim.Adam(lr=5e-5, params=[
                            {'params': net_3.hyper_net.parameters()},
                            # {'params': net.encoder.parameters()}
                        ])

for e in range(100):
  for i, (img, coords) in enumerate(loader):
      
      # model_input, ground_truth = coords# get_mgrid(28, 2), torch.Tensor(np.array(mnist_trainset[step % len(mnist_trainset)][0])).reshape((784, 1))
      # model_input, ground_truth = model_input.cuda(), ground_truth.cuda()
      img, coords = img.cuda(), coords.cuda()
      
      with torch.no_grad():
        embedding = network_head(img.reshape(-1, 1, 28, 28)).cuda() / 1000

      out = net_3({'embedding': embedding, 'coords': coords.cuda()})

      model_output = out["model_out"]
      embedding = out["latent_vec"]
      hypo_params = out["hypo_params"]

      all_hypo_params = torch.cat([x.view(-1) for x in list(hypo_params.values())])

      loss = ((model_output.cuda() - (img / 255))**2).mean() + 100 * ((all_hypo_params.cuda())**2).mean() #  + 0.1 * ((embedding.cuda())**2).mean()
      
      if not i % 250:
          print("Step %d, Total loss %0.6f" % (i, loss))

          print(torch.max(model_output))
          print(torch.max(img))

          fig, axes = plt.subplots(1,3, figsize=(18,6))
          axes[0].imshow((model_output[0]).cpu().view(28,28).detach().numpy())
          axes[1].imshow((img / 255)[0].cpu().view(28,28).detach().numpy())
          plt.show()

      optim.zero_grad()
      loss.backward()
      optim.step()

In [ ]:
torch.save(net_3.state_dict(), "mnist_siren_CNN")